In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from datetime import datetime
from scipy.integrate import simps

First thing we need to do is to estimate the risk-free rate and SOFR rate.

In [2]:
# import data
data_treasury = pd.read_csv('/Users/archibaldgonah/Desktop/Masters work/Code-Work/data/3-month treasury bill.csv')
data_sofr = pd.read_csv('/Users/archibaldgonah/Desktop/Masters work/Code-Work/data/SOFR30DAYAVG.csv')

# convert date strings to datetime objects (assuming proper date format)
data_treasury['DATE'] = pd.to_datetime(data_treasury['DATE'])
data_treasury.set_index('DATE', inplace=True)

data_sofr['DATE'] = pd.to_datetime(data_sofr['DATE'])
data_sofr.set_index('DATE', inplace=True)


In [3]:
# converting column to float
def convert_to_numeric(column):
  try:
    return float(column.replace(',', '')) 
  except ValueError:
    return None 

data_treasury['DTB3'] = data_treasury['DTB3'].apply(convert_to_numeric)
data_sofr['SOFR30DAYAVG'] = data_sofr['SOFR30DAYAVG'].apply(convert_to_numeric)


In [4]:
data_treasury.head()

,DTB3
DATE,
2019-06-12,2.19
2019-06-13,2.14
2019-06-14,2.15
2019-06-17,2.18
2019-06-18,2.17


In [5]:
data_sofr.head()

,SOFR30DAYAVG
DATE,
2019-06-13,2.40658
2019-06-14,2.40525
2019-06-17,2.39589
2019-06-18,2.39556
2019-06-19,2.39456


In [6]:
# checking data
data_treasury.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1306 entries, 2019-06-12 to 2024-06-12
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DTB3    1253 non-null   float64
dtypes: float64(1)
memory usage: 20.4 KB


In [7]:
# checking data
data_sofr.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1306 entries, 2019-06-13 to 2024-06-13
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SOFR30DAYAVG  1251 non-null   float64
dtypes: float64(1)
memory usage: 20.4 KB


In order to handle null values we are going to use Forward fill. 

This will propagate the last valid rate forward to fill the missing value. 

In [8]:
# handling nun-values

data_treasury['DTB3'] = data_treasury['DTB3'].fillna(method='ffill')
data_sofr['SOFR30DAYAVG'] = data_sofr['SOFR30DAYAVG'].fillna(method='ffill')

In [9]:
# check data
data_treasury.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1306 entries, 2019-06-12 to 2024-06-12
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DTB3    1306 non-null   float64
dtypes: float64(1)
memory usage: 20.4 KB


In [10]:
# check data
data_sofr.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1306 entries, 2019-06-13 to 2024-06-13
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SOFR30DAYAVG  1306 non-null   float64
dtypes: float64(1)
memory usage: 20.4 KB


Now lets optimize our parameters for CIR model using our historical data. We are using the CIR model to get the risk-free rate for 2024 (estimated).

In [11]:
# define the CIR model likelihood function
def cir_log_likelihood(params, rates, dt):
    kappa, theta, sigma = params
    n = len(rates)
    ll = 0.0
    
    for i in range(1, n):
        r_t = rates[i-1]
        r_t1 = rates[i]
        
    
        mean_r_t1 = r_t * np.exp(-kappa * dt) + theta * (1 - np.exp(-kappa * dt))
        var_r_t1 = (sigma**2 * r_t * (1 - np.exp(-2 * kappa * dt))) / (2 * kappa)
        
        
        var_r_t1 = max(var_r_t1, 1e-10)
        
        # log-likelihood of observing r_t1 given r_t
        ll += -0.5 * (np.log(2 * np.pi * var_r_t1) + ((r_t1 - mean_r_t1)**2) / var_r_t1)
    
    
    return -ll

# define the initial parameters and the interest rate time series
initial_params = [0.1, 0.02, 0.1]  
rates = np.array(data_treasury['DTB3'] / 100) 
dt = 1 / 252  

# perform the optimization to find the best-fit parameters
result = minimize(cir_log_likelihood, initial_params, args=(rates, dt), method='L-BFGS-B',
                  bounds=[(1e-5, None), (1e-5, None), (1e-5, None)])


kappa_opt1, theta_opt1, sigma_opt1 = result.x

print(f"Optimal parameters: kappa = {kappa_opt1}, theta = {theta_opt1}, sigma = {sigma_opt1}")


Optimal parameters: kappa = 2.3254288355309765, theta = 0.017426814468824353, sigma = 0.09867955137378603


In [12]:
rates = np.array(data_sofr['SOFR30DAYAVG'] / 100) 
dt = 1 / 252  

# perform the optimization to find the best-fit parameters
result = minimize(cir_log_likelihood, initial_params, args=(rates, dt), method='L-BFGS-B',
                  bounds=[(1e-5, None), (1e-5, None), (1e-5, None)])


kappa_opt2, theta_opt2, sigma_opt2 = result.x

print(f"Optimal parameters: kappa = {kappa_opt2}, theta = {theta_opt2}, sigma = {sigma_opt2}")


Optimal parameters: kappa = 9.608901879109297e-05, theta = 3.489258989645634, sigma = 0.0290968688172751


In [13]:
last_date = data_treasury.index[len(data_treasury.index) - 1]
future_date = datetime(2024,12,31)
difference1 = future_date - last_date

last_date = data_sofr.index[len(data_sofr.index) - 1]
future_date = datetime(2024,12,31)
difference = future_date - last_date

In [14]:
# CIR model parameters for treasury rate
mean_reversion_rate = kappa_opt1 
long_run_average = theta_opt1    
volatility = sigma_opt1          
initial_rate = data_treasury.iloc[-1][0] / 100    
time_horizon = 1        
num_steps = 144 # trading days from 2024-6-12 to 2024-12-31     


dt = time_horizon / num_steps


interest_rates1 = np.zeros(num_steps + 1)
interest_rates1[0] = initial_rate

for i in range(1, num_steps + 1):
    drift = mean_reversion_rate * (long_run_average - interest_rates1[i-1])
    diffusion = volatility * np.sqrt(interest_rates1[i-1]) * np.random.normal(scale=np.sqrt(dt))
    interest_rates1[i] = interest_rates1[i-1] + drift * dt + diffusion


avg_rate = simps(interest_rates1, dx=dt) / time_horizon

#print("Simulated Interest Rates:")
#print(interest_rates * 100)
#print("\nAverage Interest Rate:", avg_rate * 100)


In [15]:
# CIR model parameters
mean_reversion_rate = kappa_opt2 
long_run_average = theta_opt2    
volatility = sigma_opt2          
initial_rate = data_sofr.iloc[-1][0] / 100    
time_horizon = 1        
num_steps = 143 # trading days from 2024-6-13 to 2024-12-31     


dt = time_horizon / num_steps


interest_rates2 = np.zeros(num_steps + 1)
interest_rates2[0] = initial_rate

for i in range(1, num_steps + 1):
    drift = mean_reversion_rate * (long_run_average - interest_rates2[i-1])
    diffusion = volatility * np.sqrt(interest_rates2[i-1]) * np.random.normal(scale=np.sqrt(dt))
    interest_rates2[i] = interest_rates2[i-1] + drift * dt + diffusion


avg_rate = simps(interest_rates2, dx=dt) / time_horizon

#print("Simulated Interest Rates:")
#print(interest_rates * 100)
#print("\nAverage Interest Rate:", avg_rate * 100)


In [16]:
Int = np.append(np.array(data_treasury['DTB3'].loc['2024-1-1':]), np.array(interest_rates1)).mean()
r1 = np.append(np.array(data_sofr['SOFR30DAYAVG'].loc['2024-1-1':]), np.array(interest_rates2)).mean()

Now we bring in the single-period model

- This code is translated from R

In [17]:
# Set parameters
T = 1  # time period
m = 1000  # simulation times
premium = 0.03  # the extra risk premium
Int = 0.12  # estimated risk-free interest rate for 2024 (CIR)
Infl = 0.0316  # inflation rate
r1 = np.repeat(r1, m)  # estimated SOFR rate for 2024 (CIR)
K = 1000  # face value of the CAT bond

# Define parameters
c1 = 0.05866229  # Shape parameter
loc1 = 0.4833782  # Location parameter
scale1 = 4.71946946  # Scale parameter

c2 = 0.1181457  # Shape parameter
loc2 = 0.4833782  # Location parameter
scale2 = 4.9275121  # Scale parameter

# Create a frozen GEV distribution object
gev_dist1 = stats.genextreme(c1, loc1, scale1)
gev_dist2 = stats.genextreme(c2, loc2, scale2)

# Generate random variates
Mag1 = random_variates1 = gev_dist1.rvs(size=m)
Mag2 = random_variates2 = gev_dist2.rvs(size=m)
Depth1 = stats.gamma.rvs(2.35378504, 0,0.25460951 ,m )
Depth2 = stats.gamma.rvs(1.44878306, 0, 0.14585340, m)

# Set arrays for payoff function values
C = [None] * m
f = [None] * m
g = [None] * m
h = [None] * m
p = [None] * m
q = [None] * m
s = [None] * m

# Threshold levels
aa, ab, ac, ad, ae, af = 2.6, 2.8, 1.6, 1.8, 0.5, 0.6
ba, bb, bc, bd, be, bf = 2.9, 3.0, 1.8, 2.0, 1.0, 1.1
ag, ah, ai, aj, ak = 0.8, 0.85, 0.55, 0.6, 0.2
bg, bh, bi, bj, bk = 0.95, 0.98, 0.7, 0.75, 0.5

Price payoff function PCAT

In [18]:
for i in range(m):
    if Mag1[i] > Mag2[i]:
        if Mag1[i] < 5.4:
            if Depth1[i] <= 20:
                f[i] = aa * r1[i]
            if Depth1[i] > 20:
                f[i] = ab * r1[i]

            C[i] = K * (1 + f[i])

        if Mag1[i] < 5.8 and Mag1[i] >= 5.4:
            if Depth1[i] <= 15:
                g[i] = ac * r1[i]
            if Depth1[i] > 15:
                g[i] = ad * r1[i]
            
            C[i] = K * (1 +g[i])

        if Mag1[i] < 6.2 and Mag1[i] >= 5.8:
            if Depth1[i] <= 10:
                h[i] = ae * r1[i]
            if Depth1[i] > 10:
                h[i] = af * r1[i]
            
            C[i] = K * (1+ h[i])
        
        if Mag1[i] < 6.6 and Mag1[i] >= 6.2:

            C[i] = K

        if Mag1[i] < 7.0 and Mag1[i] >= 6.6:
            if Depth1[i] <= 10:
                  p[i] = ag * K
            if Depth1[i] > 10:
                p[i] = ah * K

            C[i] = p[i]

          

        if Mag1[i] < 7.4 and Mag1[i] >= 7.0:
            if Depth1[i] <= 10:
                q[i] = ai * K
            if Depth1[i] > 10:
                q[i] = aj * K
            
            C[i] = q[i]

        if Mag1[i] > 7.4:
            s[i] = ak * K

            C[i] = s[i]

    else:
        if Mag2[i] < 5.4:
            if Depth2[i] <= 20:
                f[i] = ba * r1[i]
            if Depth2[i] > 20:
                f[i] = bb *r1[i]

            C[i] = K * (1+ f[i])

        if Mag2[i] < 5.8 and Mag2[i] >= 5.4:
            if Depth2[i] <= 15:
                g[i] = bc *r1[i]
            if Depth2[i] > 15:
                g[i] = bd *r1[i]

            C[i] = K * (1 + g[i])

        if Mag2[i] < 6.2 and Mag2[i] >= 5.8:
            if Depth2[i] <= 10:
                h[i] = be * r1[i]
            if Depth2[i] > 10:
                h[i] = bf * r1[i]

            C[i] = K * (1 + h[i])

        if Mag2[i] < 6.6 and Mag2[i] >= 6.2:
            
            C[i] = K

        if Mag2[i] < 7.0 and Mag2[i] <= 6.6:
            if Depth2[i] <= 10:
                p[i] = bg * K
            if Depth2[i] > 10:
                p[i] = bh * K
            
            C[i] = p[i]

        if Mag2[i] < 7.4 and Mag2[i] <= 7.0:
            if Depth2[i] <= 10:
                q[i] = bi * K
            if Depth2[i] > 10:
                q[i] = bi * K
            
            C[i] = q[i]

        if Mag2[i] > 7.4:
            s[i] = bk * K

            C[i] = s[i]

In [19]:
# convert all None values to 0
C1 = [0 if x is None else x for x in C]

In [20]:
# Calculate discount factor
discount = (1 + Infl) * (1 + premium + Int)


# Calculate the present value of the CAT bond
P = [ x / discount for x in C1]

# Calculate the mean of P (final price of the CAT bond)
mean_P = np.mean(P)


In [21]:
mean_P

2056.2265285636877